In [25]:
import os
import pandas as pd
import numpy as np
import datetime
import dataset
from dotenv import load_dotenv
from pathlib import Path
import json

In [26]:
def create_url() -> str:
    """
    Construye la URL de conexión a la base de datos PostgreSQL
    utilizando variables de entorno.
    """
    load_dotenv()

    user = os.getenv('POSTGRES_USER', 'postgres')
    password = os.getenv('POSTGRES_PASSWORD', 'postgres')
    host = os.getenv('POSTGRES_HOST', 'localhost')
    port = os.getenv('POSTGRES_PORT', 5432)
    db = os.getenv('POSTGRES_DB', 'postgres')

    return f'postgresql://{user}:{password}@{host}:{port}/{db}'

In [27]:
def load_template(template_path: str) -> dict:
    """
    Load the template from a JSON file.
    
    Args:
        template_path (str): Path to the template JSON file
        
    Returns:
        dict: The loaded template
    """
    try:
        with open(template_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: Template file not found at {template_path}")
        raise
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON in template file {template_path}")
        raise


In [28]:
def calculate_dataframe(company_code: str, company_name: str, template_data: list) -> pd.DataFrame:
    """
    Fill the template with company data.
    
    Args:
        company_name (str): The company name to insert
        template_data (list): The template list containing systems and prompts
        
    Returns:
        pd.DataFrame: Filled template with company name
    """
    # Convert the template to a string to replace the company name
    template_str = json.dumps(template_data)
    filled_template_str = template_str.replace("'{company_name}'", f"'{company_name}'")

    df = pd.json_normalize(json.loads(filled_template_str), record_path=["assigned_prompts"], meta=["system_id", "system_name", "system_description", "system_template"])
    df = df.assign(company_code=company_code, company_name=company_name, created_at=datetime.datetime.now(), updated_at=datetime.datetime.now(), status=0, processed=False)
    
    return df



In [29]:
def create_records(): 
    template_path = "data/market_analysis_template.json"
    template_data = load_template(template_path)

    with open('data/companies.json') as f:
        data = json.load(f)

    df=pd.concat([calculate_dataframe(company_code=d['company_code'], company_name=d['company_name'], template_data=template_data) for d in data], ignore_index=True)

    db = dataset.connect(url=create_url())
    table = db['records']
    table.drop()
    table.insert_many(df.to_dict('records'))


In [31]:
def calculate_market_analysis(system_template, prompt):
    return "hello world"


In [ ]:
db = dataset.connect(url=create_url())
table = db['records']

# interar sobre los registros procesados=False
for record in table.find_one(processed=False):
    record['response'] = calculate_market_analysis(system_template=record['system_template'], prompt=record['prompt'])
    record['processed'] = True
    
    table.update(record, ['id'])   
